# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib.inline

UsageError: Line magic function `%matplotlib.inline` not found.


In [6]:
data = pd.read_csv('.\homepage_actions.csv')
data.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [10]:
# Get basic information about the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [11]:
# Summary statistics
data.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [12]:
# Unique values in each column
data.nunique()

timestamp    8188
id           6328
group           2
action          2
dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [18]:
# Separate control and experimental groups
control_group = data[data['group'] == 'control']
experiment_group = data[data['group'] == 'experiment']

# Calculate conversion rates
control_conversion_rate = control_group['id'].mean()
experiment_conversion_rate = experiment_group['id'].mean()


# # Print conversion rates
print(f'Control Conversion Rate: {control_conversion_rate}')
print(f'Experiment Conversion Rate: {experiment_conversion_rate}')

Control Conversion Rate: 562817.7164634146
Experiment Conversion Rate: 566744.8544852191


In [19]:
#Your code here
from scipy.stats import ttest_ind

# Conduct a two-sample t-test
t_stat, p_value = ttest_ind(control_group['id'], experiment_group['id'])

# Print the test statistic and p-value
print(f'Test Statistic: {t_stat}')
print(f'P-value: {p_value}')

Test Statistic: -0.8102833514715588
P-value: 0.4178008959709567


In [20]:
# Define significance level
alpha = 0.05

# Determine whether to reject the null hypothesis
if p_value < alpha:
    print("Reject the null hypothesis. The experimental homepage is significantly different from the control homepage.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between the experimental and control homepages.")

Fail to reject the null hypothesis. There is no significant difference between the experimental and control homepages.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [28]:
# #Your code here
control_click_cat = control_group[control_group['action'] == 'click']
control_ctr = control_click_cat['id'].mean()
# Size of the experiment group
experiment_group = data[data['group'] == 'experiment']
n_experiment = len(experiment_group)

# Expected number of clicks in the experiment group
expected_clicks_experiment = n_experiment * control_ctr

# Print the expected number of clicks
print(f'Expected Number of Clicks in the Experiment Group: {expected_clicks_experiment}')
# print(click_cat)

Expected Number of Clicks in the Experiment Group: 2213285495.137339


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [54]:
#Your code here
import numpy as np

# Filter the experiment group for click actions
experiment_click_cat = experiment_group[experiment_group['action'] == 'click']

# Actual number of clicks in the experiment group
actual_clicks_experiment = experiment_click_cat.shape[0]

# Standard deviation of the expected number of clicks
std_dev_clicks = n_experiment * control_ctr * (1 - control_ctr)

# Number of standard deviations (z-score)
z_score = (actual_clicks_experiment - expected_clicks_experiment) / std_dev_clicks

# Print the z-score
print(f'Z-score: {z_score}')

# print(actual_clicks_experiment)
# print(n_experiment)
# print(control_ctr)
# print(n_experiment * control_ctr * (1 - control_ctr))


Z-score: 1.772932285657916e-06


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [56]:
from scipy.stats import norm
#Your code here
# Calculate the p-value
p_value = 2 * (1 - norm.cdf(abs(z_score)))
print(f'P-value: {p_value}')

P-value: 0.999998585404702


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

both methods agree on whether there is a significant difference between the control and experiment groups.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.